<a href="https://colab.research.google.com/github/Investigator13th/CS246/blob/main/CS246_homework1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

首先搭载spark，装载云盘，导入包

In [228]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [229]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [230]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Homework1的第一部分是实现一个社交推荐系统，输入是一个图，输出是预测每个用户最有可能认识的10个朋友。


---


思路：
如何定义最有可能认识的人？


1.   a与b不是朋友，即两个node之间没有edge
2.   如果a与b有较多的共同朋友，那么这两个人可能相互认识
3.   采用共同邻居相似度衡量a与b的相似性

In [231]:
# 读数据
data = spark.read.text("/content/drive/MyDrive/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt")

In [232]:
# 解析数据，处理过滤没有朋友的用户，没有朋友的用户在无向图中是孤立点，删除没有影响
parsed_data = data.rdd.map(lambda row: row.value.split("\t")) \
                      .map(lambda x: (int(x[0]), list(map(int, x[1].split(","))) if x[1] != '' else []))\
                      .filter(lambda x: x[1]!=[])

In [233]:
# 构建用户-朋友对
user_friend_pairs = parsed_data.flatMap(lambda x: [(x[0], friend) for friend in x[1]])

In [234]:
existing_pairs = set(user_friend_pairs.collect())

In [235]:
# 使用join操作生成所有可能的用户对
# 自连接是一个很重要的步骤，通过它可以为两个用户找到一个共同邻居
# user0有朋友user1与user2，通过自连接将user1与user2连接起来，这样user1与user2的共同邻居数量就是1:((1,2),1)
# 当user(x)也同时拥有朋友user1与user2时，同样会生成一个((1,2),1)
# 用reducebykey函数聚合，就得到了user1与user2的共同邻居为2，分别是user0与user(x)
joined_pairs = user_friend_pairs.join(user_friend_pairs)
joined_pairs.take(5)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# 自连接存在问题  1.出现用户-朋友对 如(0,(1,5))其中用户1与用户5已经是朋友了，这个对应该剔除   2.出现自己的对如(1,1)
# 过滤已经存在于 user_friend_pairs 中的对并过滤掉自己的对
filtered_pairs = joined_pairs.filter(lambda x: (x[1][0], x[1][1]) not in existing_pairs).filter(lambda x: x[1][0] != x[1][1])
filtered_pairs.take(5)

In [ ]:
# 生成用户对及其共同朋友计数
user_pairs_with_common_friend = filtered_pairs.map(lambda x: ((x[1][0], x[1][1]), 1))

In [ ]:
# 聚合共同朋友计数
friend_pairs = user_pairs_with_common_friend.reduceByKey(lambda a, b: a + b)

In [ ]:
# 推荐朋友
recommendations = friend_pairs.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                              .groupByKey() \
                              .mapValues(lambda x: sorted(list(x), key=lambda y: -y[1])[:10])

In [ ]:
user_recommendations = recommendations.collectAsMap()

In [238]:
from operator import itemgetter
users_list = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]
output = itemgetter(*users_list)(user_recommendations)

In [239]:
output

([(43748, 1),
  (11860, 1),
  (15416, 1),
  (6995, 1),
  (439, 1),
  (45881, 1),
  (2409, 1)],
 [(8943, 2), (8944, 2), (8940, 1)],
 [(8939, 3), (8944, 1), (8940, 1), (8943, 1)],
 [(9022, 2), (317, 1), (9023, 1)],
 [(9021, 3), (9016, 2), (9022, 2), (9017, 2), (9023, 1), (317, 1)],
 [(9020, 3), (9017, 2), (9022, 2), (9016, 2), (9023, 1), (317, 1)],
 [(9020, 2), (9021, 2), (9019, 2), (9016, 1), (9017, 1), (9023, 1), (317, 1)],
 [(34642, 1),
  (13478, 1),
  (13134, 1),
  (37941, 1),
  (13877, 1),
  (34299, 1),
  (34485, 1)],
 [(9987, 4), (9989, 4), (35667, 3), (9991, 2)],
 [(9991, 5),
  (13877, 1),
  (34485, 1),
  (37941, 1),
  (34299, 1),
  (13478, 1),
  (34642, 1),
  (13134, 1)])

homework的难度和colab的难度真不是一个级别的，怪不得网上没有资料可以抄，就这个1-1我就做了将近两天  
还记得cs246的老师说一个homework应当花费2小时，花费超过10小时表明不适合这门课...  
令人感叹